In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# A Novel Hybrid Deep Learning Model for Aspect-based sentiment Analysis

# Component-1(LSTM)

In [3]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2025-06-01 06:28:54.177923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748759334.383350      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748759334.447800      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
data = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [6]:
data.shape

(50000, 2)

In [7]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [9]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [10]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

/tmp/ipykernel_35/2568826810.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [11]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [12]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [13]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [14]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [15]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [16]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [17]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [18]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [19]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [20]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1748759651.277680      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model

In [23]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 165s 315ms/step - accuracy: 0.7289 - loss: 0.5292 - val_accuracy: 0.8054 - val_loss: 0.4277
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 157s 313ms/step - accuracy: 0.8439 - loss: 0.3718 - val_accuracy: 0.8668 - val_loss: 0.3206
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 156s 312ms/step - accuracy: 0.8757 - loss: 0.3078 - val_accuracy: 0.8651 - val_loss: 0.3338
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 156s 311ms/step - accuracy: 0.8891 - loss: 0.2823 - val_accuracy: 0.8711 - val_loss: 0.3185
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 156s 311ms/step - accuracy: 0.9058 - loss: 0.2362 - val_accuracy: 0.8509 - val_loss: 0.3839


# Model Evaluation

In [24]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy*100}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 95ms/step - accuracy: 0.8499 - loss: 0.3733
Test Loss: 0.36915823817253113
Test Accuracy: 85.35000085830688


# Building a Predictive System

In [25]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [26]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
The sentiment of the review is: positive


# Google Translate Implementation

In [27]:
!pip install googletrans==3.1.0a0
!pip install langdetect

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=8deaa6b7c51004c7dd572df8b7c7940abeb027fa741b547e623d77a1839844ea
  Stored in directory: /root/.cache/pip/wheels/81/f2/e0/d578821d723b473d18610ea93810e4a5402463919f07e603d9
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyper

In [28]:
from googletrans import Translator
translator = Translator()

In [29]:
from langdetect import detect, detect_langs

# text = "आप कैसे हैं"
# text = "Movie achhi nhi hai"
text = "खाना अच्छा नहीं है"

language = detect(text)
print("Detected Language:", language)

# confidence scores for multiple languages
languages = detect_langs(text)
print("Possible Languages with Probabilities:", languages)
print(languages[0])

Detected Language: hi
Possible Languages with Probabilities: [hi:0.9999972581004233]
hi:0.9999972581004233


In [30]:
source = language
destination = 'en'
translated_output = translator.translate(text, src=source, dest=destination).text
print("Translated Output:", translated_output)

Translated Output: Food is not good


In [31]:
sentiment = predict_sentiment(translated_output)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
The sentiment of the review is: negative


# Implementing GAT Model

# Multilingual Dataset

In [116]:
import json
import pandas as pd

# Load JSON data
with open('/kaggle/input/train-data/train.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# List to hold flattened entries
rows = []

# Iterate over all entries
for entry in data:
    # Reconstruct the full sentence from tokens
    sentence = " ".join(entry['token'])

    # Extract aspects if present
    for aspect in entry.get('aspects', []):
        aspect_tokens = entry['token'][aspect['from']:aspect['to']]
        aspect_term = " ".join(aspect_tokens)
        rows.append({
            "sentence": sentence,
            "aspect_term": aspect_term,
            "aspect_from": aspect['from'],
            "aspect_to": aspect['to'],
            "aspect_polarity": aspect['polarity']
        })

# Convert to DataFrame
df = pd.DataFrame(rows)

# Save to CSV
df.to_csv('new_dataset.csv', index=False, encoding='utf-8')

print("CSV conversion complete. Output saved as 'new_dataset.csv'.")


CSV conversion complete. Output saved as 'new_dataset.csv'.


In [117]:
df.head(5)

,sentence,aspect_term,aspect_from,aspect_to,aspect_polarity
0,I charge it at night and skip taking the cord ...,cord,9,10,neutral
1,I charge it at night and skip taking the cord ...,battery life,16,18,positive
2,The tech guy then said the service center does...,service center,6,8,negative
3,The tech guy then said the service center does...,`` sales '' team,22,26,negative
4,The tech guy then said the service center does...,tech guy,1,3,neutral


In [118]:
df.shape

(2282, 5)

In [119]:
df.head(6)

,sentence,aspect_term,aspect_from,aspect_to,aspect_polarity
0,I charge it at night and skip taking the cord ...,cord,9,10,neutral
1,I charge it at night and skip taking the cord ...,battery life,16,18,positive
2,The tech guy then said the service center does...,service center,6,8,negative
3,The tech guy then said the service center does...,`` sales '' team,22,26,negative
4,The tech guy then said the service center does...,tech guy,1,3,neutral
5,"it is of high quality , has a killer GUI , is ...",quality,4,5,positive


In [120]:
df.tail(6)

,sentence,aspect_term,aspect_from,aspect_to,aspect_polarity
2276,We also use Paralles so we can run virtual mac...,Windows XP Professional,11,14,neutral
2277,We also use Paralles so we can run virtual mac...,", Windows 7 Home Premium",14,19,neutral
2278,We also use Paralles so we can run virtual mac...,", Windows Server Enterprise 2003",19,24,neutral
2279,We also use Paralles so we can run virtual mac...,Windows Server 2008 Enterprise,26,30,neutral
2280,"How Toshiba handles the repair seems to vary ,...",repair,25,26,positive
2281,I would like to use a different operating syst...,operating system,7,9,neutral


In [121]:
df["aspect_polarity"].value_counts()

aspect_polarity
positive    976
negative    851
neutral     455
Name: count, dtype: int64

In [122]:
df.replace({"aspect_polarity": {"positive": 1, "negative": -1, "neutral": 0}}, inplace=True)

/tmp/ipykernel_35/3147649589.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"aspect_polarity": {"positive": 1, "negative": -1, "neutral": 0}}, inplace=True)


In [123]:
df.head(6)

,sentence,aspect_term,aspect_from,aspect_to,aspect_polarity
0,I charge it at night and skip taking the cord ...,cord,9,10,0
1,I charge it at night and skip taking the cord ...,battery life,16,18,1
2,The tech guy then said the service center does...,service center,6,8,-1
3,The tech guy then said the service center does...,`` sales '' team,22,26,-1
4,The tech guy then said the service center does...,tech guy,1,3,0
5,"it is of high quality , has a killer GUI , is ...",quality,4,5,1


# Multi-language Convertions

In [124]:
!pip install deep-translator pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.3 MB/s eta 0:00:00


In [126]:
import pandas as pd
from deep_translator import GoogleTranslator
import random

# Load the dataset
df = pd.read_csv("/kaggle/input/laptop-dataset-multilingual/new_dataset.csv")  # Replace with your actual file

# Translation function
def translate_to_hindi(text):
    if isinstance(text, str) and text.strip():
        try:
            return GoogleTranslator(source='en', target='hi').translate(text)
        except Exception as e:
            print(f"Translation failed for: {text} | Error: {e}")
            return text
    return text

# Initialize columns with blank values
df['sentence_translated'] = ""
df['aspect_term_translated'] = ""

# Calculate 10% of the total rows (at least 1)
sample_size = max(1, len(df) // 10)

# Randomly select row indices to translate
rows_to_translate = random.sample(range(len(df)), sample_size)

# Apply translation to selected rows
for idx in rows_to_translate:
    df.at[idx, 'sentence_translated'] = translate_to_hindi(df.at[idx, 'sentence'])
    df.at[idx, 'aspect_term_translated'] = translate_to_hindi(df.at[idx, 'aspect_term'])

# Save to a new CSV (optional)
df.to_csv("translated_subset.csv", index=False)

# Print translated rows as a check
print(df.loc[rows_to_translate, ['sentence', 'sentence_translated', 'aspect_term', 'aspect_term_translated']])

Translation failed for: It 's still beautiful and has better color reproduction than I could ever expect from a notebook . | Error: It 's still beautiful and has better color reproduction than I could ever expect from a notebook . --> No translation was found using the current translator. Try another translator?
                                               sentence  \
1577  It gives me the power and speed that I need to...   
1722  Garageband is easy to work with , like all the...   
165   BEST BUY - 5 STARS + + + ( sales , service , r...   
1060  The only downfall is a lot of the software I h...   
2094  The 13 '' Macbook Pro just fits in my budget a...   
...                                                 ...   
58    Drivers updated ok but the BIOS update froze t...   
9     Easy to start up and does not overheat as much...   
2164  3 weeks went by and the computer keeps crashin...   
400   So what if the laptops/mobile phones look chic...   
780   The screen is bright and clear 

In [128]:
import pandas as pd
from deep_translator import GoogleTranslator

# Load the dataset
df = pd.read_csv("/kaggle/input/laptop-dataset-multilingual/new_dataset.csv")  # Replace with your file path

# Function to translate text from English to Hindi
def translate_to_hindi(text):
    if isinstance(text, str) and text.strip():
        try:
            return GoogleTranslator(source='en', target='hi').translate(text)
        except Exception as e:
            print(f"Translation error: {e} | Text: {text}")
            return text
    return text

# Translate and replace the first 2 rows
for idx in range(min(2, len(df))):
    df.at[idx, 'sentence'] = translate_to_hindi(df.at[idx, 'sentence'])
    df.at[idx, 'aspect_term'] = translate_to_hindi(df.at[idx, 'aspect_term'])

# Save the updated dataset (optional)
df.to_csv("dataset_translated_first2rows.csv", index=False)

# Display the first few rows
print(df.head())

                                            sentence       aspect_term  \
0  मैं इसे रात में चार्ज करता हूं और अच्छी बैटरी ...             घिसना   
1  मैं इसे रात में चार्ज करता हूं और अच्छी बैटरी ...      बैटरी की आयु   
2  The tech guy then said the service center does...    service center   
3  The tech guy then said the service center does...  `` sales '' team   
4  The tech guy then said the service center does...          tech guy   

   aspect_from  aspect_to aspect_polarity  
0            9         10         neutral  
1           16         18        positive  
2            6          8        negative  
3           22         26        negative  
4            1          3         neutral  


In [129]:
df.head(5)

,sentence,aspect_term,aspect_from,aspect_to,aspect_polarity
0,मैं इसे रात में चार्ज करता हूं और अच्छी बैटरी ...,घिसना,9,10,neutral
1,मैं इसे रात में चार्ज करता हूं और अच्छी बैटरी ...,बैटरी की आयु,16,18,positive
2,The tech guy then said the service center does...,service center,6,8,negative
3,The tech guy then said the service center does...,`` sales '' team,22,26,negative
4,The tech guy then said the service center does...,tech guy,1,3,neutral


In [130]:
import pandas as pd
import random
from deep_translator import GoogleTranslator

# Load the dataset
df = pd.read_csv("/kaggle/input/laptop-dataset-multilingual/new_dataset.csv")  # Replace with actual file path

# Define language codes and their names for reference (optional)
languages = {
    'hi': 'Hindi',
    'pa': 'Punjabi',
    'zh-CN': 'Chinese',
    'mr': 'Marathi',
    'it': 'Italian',
    'ja': 'Japanese'
}

# Function to translate text to the specified language
def translate_text(text, target_lang):
    if isinstance(text, str) and text.strip():
        try:
            return GoogleTranslator(source='en', target=target_lang).translate(text)
        except Exception as e:
            print(f"Error translating text: '{text}' to {target_lang} | {e}")
            return text
    return text

# Pick 1000 unique random indices from the dataset
random_indices = random.sample(range(len(df)), 1000)

# Translate the selected rows
for idx in random_indices:
    lang = random.choice(list(languages.keys()))
    df.at[idx, 'sentence'] = translate_text(df.at[idx, 'sentence'], lang)
    df.at[idx, 'aspect_term'] = translate_text(df.at[idx, 'aspect_term'], lang)

# Save the updated dataset
df.to_csv("multilingual_translated_dataset.csv", index=False)

# Optional: preview a few translated rows
print(df.loc[random_indices[:5], ['sentence', 'aspect_term']])


Error translating text: ', ran' to ja | , ran --> No translation was found using the current translator. Try another translator?
                                               sentence  \
487       彼らはジャンクであり、保証会社は恐ろしいゲートウェイコンピューターを購入しないでください。   
813   इसमें कई महान कार्यक्रम हैं, जैसे कि ILife, iP...   
1238       त्यांनी एक सोनी 'प्रमाणित' तंत्रज्ञ पाठविला.   
1146  Funzionalità come la dashboard (consente di vi...   
746   यह अभी भी काम कर रहा था, लेकिन स्क्रीन पर कुछ ...   

                   aspect_term  
487                       保証会社  
813                कार्यक्रमों  
1238  सोनी 'प्रमाणित' तंत्रज्ञ  
1146     Pannello di controllo  
746                    स्क्रीन  


# GAT Model

In [62]:
import pickle


class Vocab(object):
    def __init__(self, counter, specials=["<pad>", "<unk>"]):
        self.pad_index = 0
        self.unk_index = 1
        counter = counter.copy()
        self.itos = list(specials)
        for tok in specials:
            del counter[tok]

        # sort by frequency, then alphabetically
        words_and_frequencies = sorted(counter.items(), key=lambda tup: tup[0])
        words_and_frequencies.sort(key=lambda tup: tup[1], reverse=True)

        for word, _ in words_and_frequencies:
            self.itos.append(word)

        # stoi is simply a reverse dict for itos
        self.stoi = {tok: i for i, tok in enumerate(self.itos)}

    def __eq__(self, other):
        if self.stoi != other.stoi:
            return False
        if self.itos != other.itos:
            return False
        return True

    def __len__(self):
        return len(self.itos)

    def extend(self, v):
        words = v.itos
        for w in words:
            if w not in self.stoi:
                self.itos.append(w)
                self.stoi[w] = len(self.itos) - 1
        return self

    @staticmethod
    def load_vocab(vocab_path: str):
        with open(vocab_path, "rb") as f:
            print('Loading vocab from:', vocab_path)
            return pickle.load(f)

    def save_vocab(self, vocab_path):
        with open(vocab_path, "wb") as f:
            print('Saving vocab to:', vocab_path)
            pickle.dump(self, f)

In [65]:
import json
import os
from collections import Counter
import pickle


# Define a simple Vocab class if you don't have one
class Vocab:
    def __init__(self, counter, specials=[]):
        # self.vocab = {word: idx for idx, (word, _) in enumerate(counter.most_common())}
        # self.vocab.update({special: idx for idx, special in enumerate(specials, len(self.vocab))})
        # self.idx2word = {idx: word for word, idx in self.vocab.items()}
        self.pad_index = 0
        self.unk_index = 1
        counter = counter.copy()
        self.itos = list(specials)
        for tok in specials:
            del counter[tok]

        # sort by frequency, then alphabetically
        words_and_frequencies = sorted(counter.items(), key=lambda tup: tup[0])
        words_and_frequencies.sort(key=lambda tup: tup[1], reverse=True)

        for word, _ in words_and_frequencies:
            self.itos.append(word)

        # stoi is simply a reverse dict for itos
        self.stoi = {tok: i for i, tok in enumerate(self.itos)}
        
    def __len__(self):
        return len(self.itos)

    def __eq__(self, other):
        if self.stoi != other.stoi:
            return False
        if self.itos != other.itos:
            return False
        return True

    @staticmethod
    def load_vocab(vocab_path: str):
        with open(vocab_path, "rb") as f:
            print('Loading vocab from:', vocab_path)
            return pickle.load(f)
            
    def save_vocab(self, vocab_path):
        # with open(vocab_path, "w") as f:  # Use "w" mode to save as a text file
        #     print('Saving vocab to:', vocab_path)
        #     for word, idx in self.vocab.items():
        #         f.write(f"{word}\t{idx}\n")
        with open(vocab_path, "wb") as f:
            print('Saving vocab to:', vocab_path)
            pickle.dump(self, f)
            

def main():
    # Define paths manually (change as needed)
    data_dir = "/kaggle/input/laptop-dataset"  # Replace with your data directory in Kaggle
    vocab_dir = "/kaggle/working/vocab/laptop-dataset"  # Replace with desired output directory in Kaggle
    
    # Ensure the vocab directory exists
    if not os.path.exists(vocab_dir):
        os.makedirs(vocab_dir)

    # input files
    train_file = f"{data_dir}/train.json"
    test_file = f"{data_dir}/test.json"

    # output files
    vocab_tok_file = f"{vocab_dir}/vocab_tok.vocab"
    vocab_post_file = f"{vocab_dir}/vocab_post.vocab"
    vocab_pos_file = f"{vocab_dir}/vocab_pos.vocab"
    vocab_dep_file = f"{vocab_dir}/vocab_dep.vocab"
    vocab_pol_file = f"{vocab_dir}/vocab_pol.vocab"

    # Load files
    print("loading files...")
    train_tokens, train_pos, train_dep, train_max_len = load_tokens(train_file)
    test_tokens, test_pos, test_dep, test_max_len = load_tokens(test_file)

    # Lower tokens
    train_tokens, test_tokens = [
        [t.lower() for t in tokens] for tokens in (train_tokens, test_tokens)
    ]
    
    # Counters
    token_counter = Counter(train_tokens + test_tokens)
    pos_counter = Counter(train_pos + test_pos)
    dep_counter = Counter(train_dep + test_dep)
    max_len = max(train_max_len, test_max_len)
    post_counter = Counter(list(range(-max_len, max_len)))
    pol_counter = Counter(["positive", "negative", "neutral"])


     # Build vocab
    print("building vocab...")
    token_vocab = Vocab(token_counter, specials=["<pad>", "<unk>"])
    pos_vocab = Vocab(pos_counter, specials=["<pad>", "<unk>"])
    dep_vocab = Vocab(dep_counter, specials=["<pad>", "<unk>", "<self>"])
    post_vocab = Vocab(post_counter, specials=["<pad>", "<unk>"])
    pol_vocab = Vocab(pol_counter, specials=[])
    print(
        f"token_vocab: {len(token_vocab)}, pos_vocab: {len(pos_vocab)}, dep_vocab: {len(dep_vocab)}, "
        f"post_vocab: {len(post_vocab)}, pol_vocab: {len(pol_vocab)}"
    )

    print("dumping to files...")
    token_vocab.save_vocab(vocab_tok_file)
    pos_vocab.save_vocab(vocab_pos_file)
    dep_vocab.save_vocab(vocab_dep_file)
    post_vocab.save_vocab(vocab_post_file)
    pol_vocab.save_vocab(vocab_pol_file)
    print("all done.")

def load_tokens(filename):
    with open(filename) as infile:
        data = json.load(infile)
        tokens = []
        pos = []
        dep = []
        max_len = 0
        for d in data:
            tokens.extend(d["token"])
            pos.extend(d["pos"])
            dep.extend(d["deprel"])
            max_len = max(len(d["token"]), max_len)
    print(f"{len(tokens)} tokens from {len(data)} examples loaded from {filename}.")
    return tokens, pos, dep, max_len

if __name__ == "__main__":
    main()

loading files...
28476 tokens from 1454 examples loaded from /kaggle/input/laptop-dataset/train.json.
6510 tokens from 409 examples loaded from /kaggle/input/laptop-dataset/test.json.
building vocab...
token_vocab: 3525, pos_vocab: 46, dep_vocab: 34, post_vocab: 168, pol_vocab: 3
dumping to files...
Saving vocab to: /kaggle/working/vocab/laptop-dataset/vocab_tok.vocab
Saving vocab to: /kaggle/working/vocab/laptop-dataset/vocab_pos.vocab
Saving vocab to: /kaggle/working/vocab/laptop-dataset/vocab_dep.vocab
Saving vocab to: /kaggle/working/vocab/laptop-dataset/vocab_post.vocab
Saving vocab to: /kaggle/working/vocab/laptop-dataset/vocab_pol.vocab
all done.


In [66]:
# encoding=utf-8
import sys
# sys.path.append('../')
import torch
import random
import argparse
import numpy as np
# from vocab import Vocab
# from utils import helper
from sklearn import metrics
# from loader import ABSADataLoader
# from model import RGATABSA
# from trainer import ABSATrainer
# from load_w2v import load_pretrained_embedding

In [78]:
def get_optimizer(name, parameters, lr, l2=0):
    if name == 'sgd':
        return torch.optim.SGD(parameters, lr=lr, weight_decay=l2)
    elif name == 'adagrad':
        return torch.optim.Adagrad(parameters, lr=lr, weight_decay=l2)
    elif name == 'adam':
        return torch.optim.Adam(parameters, lr=lr, weight_decay=l2) 
    elif name == 'adamax':
        return torch.optim.Adamax(parameters, lr=lr, weight_decay=l2) 
    elif name == 'adadelta':
        return torch.optim.Adadelta(parameters, lr=lr, weight_decay=l2)
    else:
        raise Exception("Unsupported optimizer: {}".format(name))

# load args config
def load_config(filename):
    try:
        dump = torch.load(filename)
    except BaseException:
        print("[ Fail: model loading failed. ]")
    return dump['config']

In [79]:
import torch
import torch.nn.functional as F
import numpy as np

class ABSATrainer(object):
    def __init__(self, args, emb_matrix=None):
        self.args = args
        self.emb_matrix = emb_matrix
        self.model = RGATABSA(args, emb_matrix=emb_matrix)
        self.parameters = [p for p in self.model.parameters() if p.requires_grad]
        self.model.cuda()
        self.optimizer = get_optimizer(args.optim, self.parameters, args.lr)

    # Load model_state and args
    def load(self, filename):
        try:
            checkpoint = torch.load(filename)
        except BaseException:
            print("Cannot load model from {}".format(filename))
            exit()
        self.model.load_state_dict(checkpoint['model'])
        self.args = checkpoint['config']

    # Save model_state and args
    def save(self, filename):
        params = {
            'model': self.model.state_dict(),
            'config': self.args,
        }
        try:
            torch.save(params, filename)
            print("Model saved to {}".format(filename))
        except BaseException:
            print("[Warning: Saving failed... continuing anyway.]")

    def update(self, batch):
        # Convert to CUDA
        batch = [b.cuda() for b in batch]

        # Unpack inputs and label
        inputs = batch[0:8]
        label = batch[-1]

        # Step forward
        self.model.train()
        self.optimizer.zero_grad()
        logits, _ = self.model(inputs)
        loss = F.cross_entropy(logits, label, reduction='mean')
        corrects = (torch.max(logits, 1)[1].view(label.size()).data == label.data).sum()
        acc = 100.0 * float(corrects) / label.size()[0]

        # Backward
        loss.backward()
        self.optimizer.step()
        return loss.data, acc

    def predict(self, batch):
        # Convert to CUDA
        batch = [b.cuda() for b in batch]

        # Unpack inputs and label
        inputs = batch[0:8]
        label = batch[-1]

        # Forward
        self.model.eval()
        logits, g_outputs = self.model(inputs)
        loss = F.cross_entropy(logits, label, reduction='mean')
        corrects = (torch.max(logits, 1)[1].view(label.size()).data == label.data).sum()
        acc = 100.0 * float(corrects) / label.size()[0]
        predictions = np.argmax(logits.data.cpu().numpy(), axis=1).tolist()
        predprob = F.softmax(logits, dim=1).data.cpu().numpy().tolist()

        return (
            loss.data,
            acc,
            predictions,
            label.data.cpu().numpy().tolist(),
            predprob,
            g_outputs.data.cpu().numpy()
        )

    def show_error(self, batch, vocab=None):
        # Convert to CUDA
        batch = [b.cuda() for b in batch]

        # Unpack inputs and label
        inputs = batch[0:8]
        label = batch[-1]

        # Forward
        self.model.eval()
        logits, g_outputs = self.model(inputs)
        loss = F.cross_entropy(logits, label, reduction='mean')
        corrects = (torch.max(logits, 1)[1].view(label.size()).data == label.data).sum()
        acc = 100.0 * float(corrects) / label.size()[0]
        predictions = np.argmax(logits.data.cpu().numpy(), axis=1).tolist()
        predprob = F.softmax(logits, dim=1).data.cpu().numpy().tolist()

        for i in range(len(batch)):
            tokids = batch[0][i]
            aspids = batch[1][i]
            ithlabel = batch[-1][i]
            pridict = predictions[i]

            if vocab is not None:
                tok = [vocab.itos[idx] for idx in tokids]
                asp_tok = [vocab.itos[idx] for idx in aspids]
                strline = '{} {} {} {} {}'.format(
                    ' '.join(tok),
                    ' '.join(asp_tok),
                    ithlabel.item(),
                    pridict,
                    ithlabel.item() == pridict
                )
                print(strline)

        return (
            loss.data,
            acc,
            predictions,
            label.data.cpu().numpy().tolist(),
            predprob,
            g_outputs.data.cpu().numpy()
        )


In [80]:
import torch.nn as nn

In [81]:
def head_to_adj(sent_len, head, tokens, label, len_, mask, directed=False, self_loop=True):
    """
    Convert a sequence of head indexes into a 0/1 adjacency matrix and label matrix.
    """
    adj_matrix = np.zeros((sent_len, sent_len), dtype=np.float32)
    label_matrix = np.zeros((sent_len, sent_len), dtype=np.int64)

    assert not isinstance(head, list)
    tokens = tokens[:len_].tolist()
    head = head[:len_].tolist()
    label = label[:len_].tolist()

    asp_idx = [idx for idx in range(len(mask)) if mask[idx] == 1]

    for idx, h in enumerate(head):
        if idx in asp_idx:
            for k in asp_idx:
                adj_matrix[idx][k] = 1
                label_matrix[idx][k] = 2

        if h != 0:
            adj_matrix[idx, h - 1] = 1
            label_matrix[idx, h - 1] = label[idx]
        else:
            if self_loop:
                adj_matrix[idx, idx] = 1
                label_matrix[idx, idx] = 2
                continue

        if not directed:
            adj_matrix[h - 1, idx] = 1
            label_matrix[h - 1, idx] = label[idx]

        if self_loop:
            adj_matrix[idx, idx] = 1
            label_matrix[idx, idx] = 2

    return adj_matrix, label_matrix


In [82]:
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class RGATABSA(nn.Module):
    def __init__(self, args, emb_matrix=None):
        super().__init__()
        in_dim = args.hidden_dim
        self.args = args
        self.enc = ABSAEncoder(args, emb_matrix=emb_matrix)
        self.classifier = nn.Linear(in_dim, args.num_class)

    def forward(self, inputs):
        hiddens = self.enc(inputs)
        logits = self.classifier(hiddens)
        return logits, hiddens


class ABSAEncoder(nn.Module):
    def __init__(self, args, emb_matrix=None):
        super().__init__()
        self.args = args
        self.emb_matrix = emb_matrix

        # #################### Embeddings ###################
        self.emb = nn.Embedding(args.tok_size, args.emb_dim, padding_idx=0)
        if emb_matrix is not None:
            self.emb.weight = nn.Parameter(emb_matrix.cuda(), requires_grad=False)

        self.pos_emb = (
            nn.Embedding(args.pos_size, args.pos_dim, padding_idx=0)
            if args.pos_dim > 0 else None
        )
        self.post_emb = (
            nn.Embedding(args.post_size, args.post_dim, padding_idx=0)
            if args.post_dim > 0 else None
        )

        # #################### Encoder ###################
        if self.args.model.lower() in ["std", "gat"]:
            embeddings = (self.emb, self.pos_emb, self.post_emb)
            self.encoder = DoubleEncoder(args, embeddings, args.hidden_dim, args.num_layers)
        elif self.args.model.lower() == "rgat":
            self.dep_emb = (
                nn.Embedding(args.dep_size, args.dep_dim, padding_idx=0)
                if args.dep_dim > 0 else None
            )
            embeddings = (self.emb, self.pos_emb, self.post_emb, self.dep_emb)
            self.encoder = DoubleEncoder(args, embeddings, args.hidden_dim, args.num_layers, use_dep=True)
        else:
            print(f"Invalid model name {self.args.model.lower()}, it should be (std, GAT, RGAT)")
            exit(0)

        # #################### pooling and fusion modules ###################
        if self.args.pooling.lower() == "attn":
            self.attn = torch.nn.Linear(args.hidden_dim, 1)

        if self.args.output_merge.lower() != "none":
            self.inp_map = torch.nn.Linear(args.hidden_dim * 2, args.hidden_dim)

        if self.args.output_merge.lower() == "attn":
            self.out_attn_map = torch.nn.Linear(args.hidden_dim * 2, 1)
        elif self.args.output_merge.lower() == "gate":
            self.out_gate_map = torch.nn.Linear(args.hidden_dim * 2, args.hidden_dim)
        elif self.args.output_merge.lower() in ["gatenorm", "gatenorm2"]:
            self.out_gate_map = torch.nn.Linear(args.hidden_dim * 2, args.hidden_dim)
            self.out_norm = nn.LayerNorm(args.hidden_dim)
        elif self.args.output_merge.lower() == "addnorm":
            self.out_norm = nn.LayerNorm(args.hidden_dim)
        elif self.args.output_merge.lower() not in ["none", "add"]:
            print("Invalid output_merge type: ", self.args.output_merge)
            exit()

        if self.args.output_merge.lower() != "none":
            self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.eye_(self.inp_map.weight)
        torch.nn.init.zeros_(self.inp_map.bias)

    def forward(self, inputs):
        tok, asp, pos, head, deprel, post, mask_ori, lengths = inputs
        maxlen = max(lengths.data)

        adj_lst, label_lst = [], []
        for idx in range(len(lengths)):
            adj_i, label_i = head_to_adj(
                maxlen, head[idx], tok[idx], deprel[idx], lengths[idx], mask_ori[idx],
                directed=self.args.direct, self_loop=self.args.loop
            )
            adj_lst.append(adj_i.reshape(1, maxlen, maxlen))
            label_lst.append(label_i.reshape(1, maxlen, maxlen))

        adj = np.concatenate(adj_lst, axis=0)
        adj = torch.from_numpy(adj).cuda()

        labels = np.concatenate(label_lst, axis=0)
        label_all = torch.from_numpy(labels).cuda()

        if self.args.model.lower() == "std":
            sent_out, graph_out = self.encoder(adj=None, inputs=inputs, lengths=lengths)
        elif self.args.model.lower() == "gat":
            sent_out, graph_out = self.encoder(adj=adj, inputs=inputs, lengths=lengths)
        elif self.args.model.lower() == "rgat":
            sent_out, graph_out = self.encoder(adj=adj, relation_matrix=label_all, inputs=inputs, lengths=lengths)
        elif self.args.model.lower() == "rgat-noadj":
            sent_out, graph_out = self.encoder(adj=None, relation_matrix=label_all, inputs=inputs, lengths=lengths)
        else:
            print(f"Invalid model name {self.args.model.lower()}, it should be (std, GAT, RGAT)")
            exit(0)

        # ########### pooling and fusion #################
        asp_wn = mask_ori.sum(dim=1).unsqueeze(-1)
        mask = mask_ori.unsqueeze(-1).repeat(1, 1, self.args.hidden_dim)

        if self.args.pooling.lower() == "avg":
            graph_out = (graph_out * mask).sum(dim=1) / asp_wn
        elif self.args.pooling.lower() == "max":
            graph_out = torch.max(graph_out * mask, dim=1).values

        if self.args.output_merge.lower() == "none":
            return graph_out

        sent_out = self.inp_map(sent_out)
        if self.args.pooling.lower() == "avg":
            sent_out = (sent_out * mask).sum(dim=1) / asp_wn
        elif self.args.pooling.lower() == "max":
            sent_out = torch.max(sent_out * mask, dim=1).values

        if self.args.output_merge.lower() == "gate":
            gate = torch.sigmoid(self.out_gate_map(torch.cat([graph_out, sent_out], dim=-1)))
            outputs = graph_out * gate + (1 - gate) * sent_out
        elif self.args.output_merge.lower() == "gatenorm":
            gate = torch.sigmoid(self.out_gate_map(torch.cat([graph_out, sent_out], dim=-1)))
            outputs = self.out_norm(graph_out * gate + (1 - gate) * sent_out)
        elif self.args.output_merge.lower() == "gatenorm2":
            gate = self.out_norm(torch.sigmoid(self.out_gate_map(torch.cat([graph_out, sent_out], dim=-1))))
            outputs = graph_out * gate + (1 - gate) * sent_out
        elif self.args.output_merge.lower() == "addnorm":
            outputs = self.out_norm(graph_out + sent_out)
        elif self.args.output_merge.lower() == "add":
            outputs = graph_out + sent_out
        elif self.args.output_merge.lower() == "attn":
            att = torch.sigmoid(self.out_attn_map(torch.cat([graph_out, sent_out], dim=-1)))
            outputs = graph_out * att + (1 - att) * sent_out

        return outputs


class DoubleEncoder(nn.Module):
    def __init__(self, args, embeddings, mem_dim, num_layers, use_dep=False):
        super(DoubleEncoder, self).__init__()
        self.args = args
        self.layers = num_layers
        self.mem_dim = mem_dim
        self.in_dim = args.emb_dim + args.post_dim + args.pos_dim

        if use_dep:
            self.emb, self.pos_emb, self.post_emb, self.dep_emb = embeddings
        else:
            self.emb, self.pos_emb, self.post_emb = embeddings

        input_size = self.in_dim
        self.Sent_encoder = nn.LSTM(
            input_size,
            args.rnn_hidden,
            args.rnn_layers,
            batch_first=True,
            dropout=args.rnn_dropout,
            bidirectional=args.bidirect,
        )
        self.in_dim = args.rnn_hidden * 2 if args.bidirect else args.rnn_hidden

        self.rnn_drop = nn.Dropout(args.rnn_dropout)
        self.in_drop = nn.Dropout(args.input_dropout)

        if use_dep:
            self.graph_encoder = RGATEncoder(
                num_layers=num_layers,
                d_model=args.rnn_hidden * 2,
                heads=args.attn_heads,
                d_ff=args.rnn_hidden * 2,
                dropout=args.layer_dropout,
                att_drop=args.att_dropout,
                use_structure=True,
                alpha=args.alpha,
                beta=args.beta,
            )
        else:
            encoder_layer = nn.TransformerEncoderLayer(
                d_model=args.rnn_hidden * 2,
                nhead=args.attn_heads,
                dim_feedforward=args.rnn_hidden * 2,
                dropout=args.layer_dropout,
            )
            self.graph_encoder = nn.TransformerEncoder(encoder_layer=encoder_layer, num_layers=num_layers)

        self.out_map = nn.Linear(args.rnn_hidden * 2, args.rnn_hidden)

    def encode_with_rnn(self, rnn_inputs, seq_lens, batch_size):
        h0, c0 = rnn_zero_state(batch_size, self.args.rnn_hidden, self.args.rnn_layers, self.args.bidirect)
        rnn_inputs = nn.utils.rnn.pack_padded_sequence(rnn_inputs, seq_lens.cpu(), batch_first=True)
        rnn_outputs, _ = self.Sent_encoder(rnn_inputs, (h0, c0))
        rnn_outputs, _ = nn.utils.rnn.pad_packed_sequence(rnn_outputs, batch_first=True)
        return rnn_outputs

    def forward(self, adj, inputs, lengths, relation_matrix=None):
        tok, asp, pos, head, deprel, post, a_mask, seq_len = inputs

        word_embs = self.emb(tok)
        embs = [word_embs]
        if self.args.pos_dim > 0:
            embs += [self.pos_emb(pos)]
        if self.args.post_dim > 0:
            embs += [self.post_emb(post)]
        embs = torch.cat(embs, dim=2)
        embs = self.in_drop(embs)

        sent_output = self.rnn_drop(self.encode_with_rnn(embs, seq_len, tok.size(0)))

        mask = adj.eq(0) if adj is not None else None
        key_padding_mask = sequence_mask(lengths) if lengths is not None else None
        dep_relation_embs = self.dep_emb(relation_matrix) if relation_matrix is not None else None

        inp = sent_output.transpose(0, 1)
        graph_output = self.graph_encoder(inp, mask=mask, src_key_padding_mask=key_padding_mask)
        graph_output = graph_output.transpose(0, 1)
        graph_output = self.out_map(graph_output)
        return sent_output, graph_output


def rnn_zero_state(batch_size, hidden_dim, num_layers, bidirectional=True):
    total_layers = num_layers * 2 if bidirectional else num_layers
    state_shape = (total_layers, batch_size, hidden_dim)
    h0 = c0 = Variable(torch.zeros(*state_shape), requires_grad=False)
    return h0.cuda(), c0.cuda()


def sequence_mask(lengths, max_len=None):
    batch_size = lengths.numel()
    max_len = max_len or lengths.max()
    return torch.arange(0, max_len, device=lengths.device).type_as(lengths).unsqueeze(0).expand(
        batch_size, max_len
    ) >= lengths.unsqueeze(1)


In [83]:
import os
import json
import numpy as np
import torch


class ABSADataLoader(object):
    def __init__(self, filename, batch_size, args, vocab, shuffle=True):
        self.batch_size = batch_size
        self.args = args
        self.vocab = vocab

        assert os.path.exists(filename), filename
        with open(filename, "r") as infile:
            data = json.load(infile)
        self.raw_data = data

        # preprocess data
        data = self.preprocess(data, vocab, args)

        print("{} instances loaded from {}".format(len(data), filename))

        if shuffle:
            indices = np.arange(len(data))
            np.random.shuffle(indices)
            data = [data[idx] for idx in indices]

        # labels
        pol_vocab = vocab[-1]
        self.labels = [pol_vocab.itos[d[-1]] for d in data]

        # example num
        self.num_examples = len(data)

        # chunk into batches
        data = [data[i: i + batch_size] for i in range(0, len(data), batch_size)]
        self.data = data
        print("{} batches created for {}".format(len(data), filename))

    def preprocess(self, data, vocab, args):
        # unpack vocab
        token_vocab, post_vocab, pos_vocab, dep_vocab, pol_vocab = vocab
        processed = []

        for d in data:
            for aspect in d["aspects"]:
                # word token
                tok = list(d["token"])
                if args.lower:
                    tok = [t.lower() for t in tok]

                # aspect
                asp = list(aspect["term"])
                # label
                label = aspect["polarity"]
                # pos_tag
                pos = list(d["pos"])
                # head
                head = list(d["head"])
                # deprel
                deprel = list(d["deprel"])
                # real length
                length = len(tok)

                # position
                post = (
                    [i - aspect["from"] for i in range(aspect["from"])]
                    + [0 for _ in range(aspect["from"], aspect["to"])]
                    + [i - aspect["to"] + 1 for i in range(aspect["to"], length)]
                )

                # aspect mask
                if len(asp) == 0:
                    mask = [1 for _ in range(length)]  # for rest16
                else:
                    mask = (
                        [0 for _ in range(aspect["from"])]
                        + [1 for _ in range(aspect["from"], aspect["to"])]
                        + [0 for _ in range(aspect["to"], length)]
                    )

                # mapping token
                tok = [token_vocab.stoi.get(t, token_vocab.unk_index) for t in tok]
                # mapping aspect
                asp = [token_vocab.stoi.get(t, token_vocab.unk_index) for t in asp]
                # mapping label
                label = pol_vocab.stoi.get(label)
                # mapping pos
                pos = [pos_vocab.stoi.get(t, pos_vocab.unk_index) for t in pos]
                # mapping head to int
                head = [int(x) for x in head]
                assert any([x == 0 for x in head])
                # mapping deprel
                deprel = [dep_vocab.stoi.get(t, dep_vocab.unk_index) for t in deprel]
                # mapping post
                post = [post_vocab.stoi.get(t, post_vocab.unk_index) for t in post]

                assert (
                    len(tok) == length
                    and len(pos) == length
                    and len(head) == length
                    and len(deprel) == length
                    and len(post) == length
                    and len(mask) == length
                )

                processed += [(tok, asp, pos, head, deprel, post, mask, length, label)]

        return processed

    def gold(self):
        return self.labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, key):
        if not isinstance(key, int):
            raise TypeError
        if key < 0 or key >= len(self.data):
            raise IndexError

        batch = self.data[key]
        batch_size = len(batch)
        batch = list(zip(*batch))

        # sort all fields by lens for easy RNN operations
        lens = [len(x) for x in batch[0]]
        batch, orig_idx = sort_all(batch, lens)

        # convert to tensors
        tok = get_long_tensor(batch[0], batch_size)
        asp = get_long_tensor(batch[1], batch_size)
        pos = get_long_tensor(batch[2], batch_size)
        head = get_long_tensor(batch[3], batch_size)
        deprel = get_long_tensor(batch[4], batch_size)
        post = get_long_tensor(batch[5], batch_size)
        mask = get_float_tensor(batch[6], batch_size)
        length = torch.LongTensor(batch[7])
        label = torch.LongTensor(batch[8])

        return (tok, asp, pos, head, deprel, post, mask, length, label)

    def __iter__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)


def get_long_tensor(tokens_list, batch_size):
    """Convert list of list of tokens to a padded LongTensor."""
    token_len = max(len(x) for x in tokens_list)
    tokens = torch.LongTensor(batch_size, token_len).fill_(0)
    for i, s in enumerate(tokens_list):
        tokens[i, :len(s)] = torch.LongTensor(s)
    return tokens


def get_float_tensor(tokens_list, batch_size):
    """Convert list of list of tokens to a padded FloatTensor."""
    token_len = max(len(x) for x in tokens_list)
    tokens = torch.FloatTensor(batch_size, token_len).fill_(0)
    for i, s in enumerate(tokens_list):
        tokens[i, :len(s)] = torch.FloatTensor(s)
    return tokens


def sort_all(batch, lens):
    """Sort all fields by descending order of lens, and return the original indices."""
    unsorted_all = [lens] + [range(len(lens))] + list(batch)
    sorted_all = [list(t) for t in zip(*sorted(zip(*unsorted_all), reverse=True))]
    return sorted_all[2:], sorted_all[1]


In [84]:
def load_pretrained_embedding(glove_dir, word_list, dimension_size=300, encoding="utf-8"):
    pre_words = []
    count = 0

    with open(glove_dir + "/glove.840B.300d.txt", "r", encoding=encoding) as fopen:
        for line in fopen:
            pre_words.append(line.strip())
    word2offset = {w: i for i, w in enumerate(pre_words)}

    word_vectors = []
    for word in word_list:
        if word in word2offset:
            line = linecache.getline(glove_dir + "/glove.840B.300d.txt", word2offset[word] + 1)
            assert word == line[: line.find(" ")].strip()
            word_vectors.append(
                np.fromstring(line[line.find(" "):].strip(), sep=" ", dtype=np.float32)
            )
            count += 1
        else:
            # init zero
            word_vectors.append(np.zeros(dimension_size, dtype=np.float32))
    print("Loading {}/{} words from vocab...".format(count, len(word_list)))

    return word_vectors

In [85]:
import os
import six

def ensure_dir(d, verbose=True):
    if not os.path.exists(d):
        if verbose:
            print("Directory {} does not exist; creating...".format(d))
        os.makedirs(d)


class FileLogger(object):
    """
    A file logger that opens the file periodically and writes to it.
    """
    def __init__(self, filename, header=None):
        self.filename = filename
        if os.path.exists(filename):
            # remove the old file
            os.remove(filename)
        if header is not None:
            with open(filename, 'w') as out:
                print(header, file=out)
    
    def log(self, message):
        with open(self.filename, 'a') as out:
            print(message, file=out)


def print_arguments(args):
    print('-----------  Configuration Arguments -----------')
    for arg, value in sorted(six.iteritems(vars(args))):
        print('%s: %s' % (arg, value))
    print('------------------------------------------------')


def unpack_raw_data(raw_data, batch_size=32):
    unpacked = []
    for d in raw_data: 
        for a in d['aspects']:
            unpacked.append({'token': d['token'], 'aspect': a, 'polarity': a['polarity']})
    
    batches = [unpacked[i:i + batch_size] for i in range(0, len(unpacked), batch_size)]
    
    sorted_unpacked = []
    for batch in batches:
        lens = [len(x['token']) for x in batch]
        temp = [t[0] for t in sorted(zip(batch, lens, range(len(lens))), key=lambda x: (x[1], x[2]), reverse=True)]
        sorted_unpacked.extend(temp)
    
    return sorted_unpacked


In [96]:
# Simulate argparse arguments for Jupyter Notebook
args = argparse.Namespace(
    data_dir="/kaggle/input/laptop-dataset",
    vocab_dir="/kaggle/working/vocab/laptop-dataset",
    glove_dir="/kaggle/input/glovewordembeddings",
    emb_dim=300,
    post_dim=30,
    pos_dim=30,
    dep_dim=30,
    hidden_dim=50,
    num_layers=2,
    num_class=3,
    cross_val_fold=10,
    input_dropout=0.7,
    layer_dropout=0,
    att_dropout=0,
    attn_heads=5,
    alpha=1.0,
    beta=1.0,
    lower=True,
    direct=False,
    bidirect=True,
    loop=True,
    rnn_hidden=50,
    rnn_layers=1,
    rnn_dropout=0.1,
    lr=0.01,
    optim="adamax",
    num_epoch=100,
    batch_size=32,
    log_step=20,
    log="logs.txt",
    save_dir="./saved_models",
     model="std",
    pooling="avg",
    output_merge="none",
    shuffle=False,
    seed=0,
    tune=False
)

# Now you can access the arguments through `args` object, e.g.:
print(args.data_dir)
print(args.emb_dim)
print(f"Training file path: {args.data_dir + '/train.json'}")
print(f"Validation file path: {args.data_dir + '/valid.json'}")
print(f"Test file path: {args.data_dir + '/test.json'}")
def get_dataloaders(args, vocab):
    train_batch = ABSADataLoader(
        args.data_dir + "/valid.json", args.batch_size, args, vocab, shuffle=args.shuffle
    )
    valid_batch = ABSADataLoader(
        args.data_dir + "/valid.json", args.batch_size, args, vocab, shuffle=False
    )
    test_batch = ABSADataLoader(
        args.data_dir + "/test.json", args.batch_size, args, vocab, shuffle=False
    )
    return train_batch, valid_batch, test_batch

/kaggle/input/laptop-dataset
300
Training file path: /kaggle/input/laptop-dataset/train.json
Validation file path: /kaggle/input/laptop-dataset/valid.json
Test file path: /kaggle/input/laptop-dataset/test.json


In [97]:
# load vocab
print("Loading vocab...")
vocab_dir="/kaggle/working/vocab/laptop-dataset"
token_vocab = Vocab.load_vocab(vocab_dir + "/vocab_tok.vocab")  # token
post_vocab = Vocab.load_vocab(vocab_dir + "/vocab_post.vocab")  # position
pos_vocab = Vocab.load_vocab(vocab_dir + "/vocab_pos.vocab")  # POS
dep_vocab = Vocab.load_vocab(vocab_dir + "/vocab_dep.vocab")  # deprel
pol_vocab = Vocab.load_vocab(vocab_dir + "/vocab_pol.vocab")  # polarity
vocab = (token_vocab, post_vocab, pos_vocab, dep_vocab, pol_vocab)
print(
    "token_vocab: {}, post_vocab: {}, pos_vocab: {}, dep_vocab: {}, pol_vocab: {}".format(
        len(token_vocab), len(post_vocab), len(pos_vocab), len(dep_vocab), len(pol_vocab)
    )
)

Loading vocab...
Loading vocab from: /kaggle/working/vocab/laptop-dataset/vocab_tok.vocab
Loading vocab from: /kaggle/working/vocab/laptop-dataset/vocab_post.vocab
Loading vocab from: /kaggle/working/vocab/laptop-dataset/vocab_pos.vocab
Loading vocab from: /kaggle/working/vocab/laptop-dataset/vocab_dep.vocab
Loading vocab from: /kaggle/working/vocab/laptop-dataset/vocab_pol.vocab
token_vocab: 3525, post_vocab: 168, pos_vocab: 46, dep_vocab: 34, pol_vocab: 3


In [98]:
print(f"Training file path: {args.data_dir + '/train.json'}")
print(f"Validation file path: {args.data_dir + '/valid.json'}")
print(f"Test file path: {args.data_dir + '/test.json'}")
def get_dataloaders(args, vocab):
    train_batch = ABSADataLoader(
        args.data_dir + "/valid.json", args.batch_size, args, vocab, shuffle=args.shuffle
    )
    valid_batch = ABSADataLoader(
        args.data_dir + "/valid.json", args.batch_size, args, vocab, shuffle=False
    )
    test_batch = ABSADataLoader(
        args.data_dir + "/test.json", args.batch_size, args, vocab, shuffle=False
    )
    return train_batch, valid_batch, test_batch

Training file path: /kaggle/input/laptop-dataset/train.json
Validation file path: /kaggle/input/laptop-dataset/valid.json
Test file path: /kaggle/input/laptop-dataset/test.json


In [99]:
def evaluate(model, data_loader):
    predictions, labels = [], []
    val_loss, val_acc, val_step = 0.0, 0.0, 0
    for i, batch in enumerate(data_loader):
        loss, acc, pred, label, _, _ = model.predict(batch)
        val_loss += loss
        val_acc += acc
        predictions += pred
        labels += label
        val_step += 1
    # f1 score
    f1_score = metrics.f1_score(labels, predictions, average="macro")
    return val_loss / val_step, val_acc / val_step, f1_score

In [100]:
def _totally_parameters(model):  #
    n_params = sum([p.nelement() for p in model.parameters()])
    return n_params

In [105]:
# train model
def trainmodel(config=None):
    if config is not None:
        args.batch_size = config["bsz"]
        args.seed = config["npseed"]
        args.npseed = config["npseed"]
        args.input_dropout = config["inp_drop"]

    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    random.seed(args.seed)
    torch.cuda.manual_seed(args.seed)

    print_arguments(args)

    train_batch, valid_batch, test_batch = get_dataloaders(args, vocab)

    trainer = ABSATrainer(args, emb_matrix=word_emb)
    print(trainer.model)
    print("Total parameters:", _totally_parameters(trainer.model))

    best_path = args.save_dir
    ensure_dir(best_path, verbose=True)

    print("Training Set: {}".format(len(train_batch)))
    print("Valid Set: {}".format(len(valid_batch)))
    print("Test Set: {}".format(len(test_batch)))

    train_acc_history, train_loss_history = [], []
    val_acc_history, val_loss_history, val_f1_score_history = [0.0], [0.0], [0.0]
    patience = 0
    epoch = 0

    for _ in range(1, args.num_epoch + 1):
        epoch += 1
        print("Epoch {}".format(epoch) + "-" * 60)
        train_loss, train_acc, train_step = 0.0, 0.0, 0

        for i, batch in enumerate(train_batch):
            loss, acc = trainer.update(batch)
            train_loss += loss
            train_acc += acc
            train_step += 1

            if train_step % args.log_step == 0:
                print(
                    "{}/{} train_loss: {:.6f}, train_acc: {:.6f}".format(
                        i, len(train_batch), train_loss / train_step, train_acc / train_step
                    )
                )

        val_loss, val_acc, val_f1 = evaluate(trainer, valid_batch)

        print(
            "End of {} train_loss: {:.4f}, train_acc: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}, f1_score: {:.4f}".format(
                epoch,
                train_loss / train_step,
                train_acc / train_step,
                val_loss,
                val_acc,
                val_f1,
            )
        )

        train_acc_history.append(train_acc / train_step)
        train_loss_history.append(train_loss / train_step)
        val_loss_history.append(val_loss)

        # save best model
        if epoch == 1 or float(val_acc) > max(val_acc_history):
            patience = 0
            torch.save(trainer, best_path + '/best_checkpoint.pt')
            print("new best model saved.")
        else:
            patience += 1

        val_acc_history.append(float(val_acc))
        val_f1_score_history.append(val_f1)

        if patience >= 20:
            print('Reach the max patience, stopping...')
            break

    print("Training ended with {} epochs.".format(epoch))

    print("Loading best checkpoints from", best_path + '/best_checkpoint.pt')
    # trainer = torch.load(best_path + '/best_checkpoint.pt')
    trainer = torch.load(best_path + '/best_checkpoint.pt', weights_only=False)
    test_loss, test_acc, test_f1 = evaluate(trainer, test_batch)
    print("Evaluation Results: test_loss:{}, test_acc:{}, test_f1:{}".format(test_loss, test_acc, test_f1))


In [106]:
args.tok_size = len(token_vocab)
args.post_size = len(post_vocab)
args.pos_size = len(pos_vocab)
args.dep_size = len(dep_vocab)

In [107]:
import os
import json
import six
import argparse
import subprocess

def ensure_dir(d, verbose=True):
    if not os.path.exists(d):
        if verbose:
            print("Directory {} do not exist; creating...".format(d))
        os.makedirs(d)

class FileLogger(object):
    """
    A file logger that opens the file periodically and writes to it.
    """
    def __init__(self, filename, header=None):
        self.filename = filename
        if os.path.exists(filename):
            # remove the old file
            os.remove(filename)
        if header is not None:
            with open(filename, 'w') as out:
                print(header, file=out)
    
    def log(self, message):
        with open(self.filename, 'a') as out:
            print(message, file=out)

def print_arguments(args):
    print('-----------  Configuration Arguments -----------')
    for arg, value in sorted(six.iteritems(vars(args))):
        print('%s: %s' % (arg, value))
    print('------------------------------------------------')

def unpack_raw_data(raw_data, batch_size=32):
    unpacked = []
    for d in raw_data: 
        for a in d['aspects']:
            unpacked.append({'token': d['token'], 'aspect': a, 'polarity': a['polarity']})
    
    batches = [unpacked[i:i+batch_size] for i in range(0, len(unpacked), batch_size)]
    
    unpacked = []
    for batch in batches:
        lens = [len(x['token']) for x in batch]
        temp = [t[0] for t in list(sorted(zip(batch, lens, list(range(len(lens)))), key=lambda x: (x[1], x[2]), reverse=True))]
        unpacked.extend(temp)
    
    return unpacked


In [108]:
# load pretrained word emb
print("Loading pretrained word emb...")
word_emb = load_pretrained_embedding(glove_dir="/kaggle/input/glovewordembeddings", word_list=token_vocab.itos)
assert len(word_emb) == len(token_vocab)
assert len(word_emb[0]) == args.emb_dim
word_emb = torch.FloatTensor(word_emb)  # convert to tensor


if __name__ == "__main__":
    trainmodel()

Loading pretrained word emb...
Loading 0/3525 words from vocab...
-----------  Configuration Arguments -----------
alpha: 1.0
att_dropout: 0
attn_heads: 5
batch_size: 32
beta: 1.0
bidirect: True
cross_val_fold: 10
data_dir: /kaggle/input/laptop-dataset
dep_dim: 30
dep_size: 34
direct: False
emb_dim: 300
glove_dir: /kaggle/input/glovewordembeddings
hidden_dim: 50
input_dropout: 0.7
layer_dropout: 0
log: logs.txt
log_step: 20
loop: True
lower: True
lr: 0.01
model: std
num_class: 3
num_epoch: 100
num_layers: 2
optim: adamax
output_merge: none
pooling: avg
pos_dim: 30
pos_size: 46
post_dim: 30
post_size: 168
rnn_dropout: 0.1
rnn_hidden: 50
rnn_layers: 1
save_dir: ./saved_models
seed: 0
shuffle: False
tok_size: 3525
tune: False
vocab_dir: /kaggle/working/vocab/laptop-dataset
------------------------------------------------
632 instances loaded from /kaggle/input/laptop-dataset/valid.json
20 batches created for /kaggle/input/laptop-dataset/valid.json
632 instances loaded from /kaggle/input/l

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


19/20 train_loss: 1.278596, train_acc: 43.437500
End of 1 train_loss: 1.2786, train_acc: 43.4375, val_loss: 1.2468, val_acc: 26.5625, f1_score: 0.1393
new best model saved.
Epoch 2------------------------------------------------------------
19/20 train_loss: 1.154354, train_acc: 45.468750
End of 2 train_loss: 1.1544, train_acc: 45.4688, val_loss: 1.1633, val_acc: 26.5625, f1_score: 0.1393
Epoch 3------------------------------------------------------------
19/20 train_loss: 1.110446, train_acc: 37.968750
End of 3 train_loss: 1.1104, train_acc: 37.9688, val_loss: 1.0206, val_acc: 53.2812, f1_score: 0.2319
new best model saved.
Epoch 4------------------------------------------------------------
19/20 train_loss: 1.024956, train_acc: 53.281250
End of 4 train_loss: 1.0250, train_acc: 53.2812, val_loss: 1.0131, val_acc: 53.2812, f1_score: 0.2319
Epoch 5------------------------------------------------------------
19/20 train_loss: 1.025551, train_acc: 53.281250
End of 5 train_loss: 1.0256, tr